In [ ]:
from transformers import AutoTokenizer, MixtralModel

model = MixtralModel.from_pretrained("hf-internal-testing/Mixtral-tiny")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mixtral-8x7B-v0.1")

prompt = "Hey, are you conscious? Can you talk to me?"
inputs = tokenizer(prompt, return_tensors="pt")

x = model(**inputs)
x

In [ ]:
from transformers import AutoTokenizer, FlaxMixtralModel

model = FlaxMixtralModel.from_pretrained("hf-internal-testing/Mixtral-tiny")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mixtral-8x7B-v0.1")

prompt = "Hey, are you conscious? Can you talk to me?"
inputs = tokenizer(prompt, return_tensors="jax")

x = model(**inputs)
x

In [ ]:
# from transformers import AutoTokenizer, MixtralForCausalLM

# model = MixtralForCausalLM.from_pretrained("hf-internal-testing/Mixtral-tiny")
# tokenizer = AutoTokenizer.from_pretrained("mistralai/Mixtral-8x7B-v0.1")

# prompt = "Hey, are you conscious? Can you talk to me?"
# inputs = tokenizer(prompt, return_tensors="pt")

# # Generate
# generate_ids = model.generate(inputs.input_ids, max_length=30)
# tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

In [ ]:
# from transformers import AutoTokenizer, FlaxMixtralForCausalLM

# model = FlaxMixtralForCausalLM.from_pretrained("hf-internal-testing/Mixtral-tiny")
# tokenizer = AutoTokenizer.from_pretrained("mistralai/Mixtral-8x7B-v0.1")

# prompt = "Hey, are you conscious? Can you talk to me?"
# inputs = tokenizer(prompt, return_tensors="jax")

# # Generate
# generate_ids = model.generate(inputs.input_ids, max_length=30)
# tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]